In [2]:
import numpy
import numpy as np
import pandas
import pandas as pd
from numba import njit
import sys,glob
import zipfile, os, time
import urllib3
from datetime import datetime, timedelta
import glob
import h5py
import plotly.express as px
sys.path.append(os.getcwd()+'/../cds-backend/code/')
sys.path.append(os.getcwd()+'/../harvest/code/')
from harvest_convert_to_netCDF_newfixes import write_dict_h5
import cds_eua3 as eua
eua.logging_set_level(30)
import xarray as xr

import cdsapi, zipfile, os, time
#import schedule
import copy
from shutil import copyfile
import multiprocessing
sys.path.append(os.getcwd()+'/rasotools-master/')
import rasotools

In [ ]:
rasotools.met.

    34	humidity	atmospheric	surface; upper-air	dew point depression	K	Dew point depression is also called dew point deficit. It is the amount by which the air temperature exceeds its dew point temperature. Dew point temperature is the temperature at which a parcel of air reaches saturation upon being cooled at constant pressure and specific humidity.

    36	humidity	atmospheric	surface; upper-air	dew point temperature	K	Dew point temperature is the temperature at which a parcel of air reaches saturation upon being cooled at constant pressure and specific humidity.

    37	humidity	atmospheric	surface; upper-air	ice bulb temperature	K	TBD

    38	humidity	atmospheric	surface; upper-air	relative humidity	percent	TBD

    39	humidity	atmospheric	surface; upper-air	specific humidity	g kg-1	specific means per unit mass. Specific humidity is the mass fraction of water vapor in (moist) air.



    104	wind	atmospheric	surface; upper-air	eastward wind speed	m s-1	Eastward indicates a vector component which is positive when directed eastward (negative westward). Wind is defined as a two-dimensional (horizontal) air velocity vector,  with no vertical component. (Vertical motion in the atmosphere has the standard name upward air velocity.)

    105	wind	atmospheric	surface; upper-air	northward wind speed	m s-1	Northward indicates a vector component which is positive when directed northward (negative southward). Wind is defined as a two-dimensional (horizontal) air velocity vector,  with no vertical component. (Vertical motion in the atmosphere has the standard name upward air velocity.)

    106	wind	atmospheric	surface; upper-air	wind from direction	degree	direction from which the wind is blowing Lot 1 uses dd  - WMO abbrev.

    107	wind	atmospheric	surface; upper-air	wind speed	m s-1	Speed is the magnitude of velocity. Wind is defined as a two-dimensional (horizontal) air velocity vector,  with no vertical component. (Vertical motion in the atmosphere has the standard name upward air velocity.) The wind speed is the magnitude of the wind velocity. Lot 1 uses ff  - WMO abbrev.

    137	humidity	atmospheric	surface; upper air	air dewpoint	K	Dewpoint measurement (from profile measurement)

    138	humidity	atmospheric	surface; upper air	relative humidity	1	Relative humidity (from profile measurement)

    139	wind	atmospheric	surface; upper air	eastward wind speed	m s-1	Eastward wind speed (from profile measurement)

    140	wind	atmospheric	surface; upper air	northward wind speed	m s-1	Northward wind speed (from profile measurement)

In [3]:
files = glob.glob('/raid60/scratch/uli/resorted_files_correct/*.nc')

In [8]:
def check_for_missing(file, checkvars: list = [85, 38, 106, 107], convert: bool = False):
    missing = []
    with eua.CDMDataset(file) as data:
        for i in checkvars:
            if not str(i) in data.recordindices.keys():
                missing.append(i)
        if not convert:
            return missing
        else: 
            convert_missing(file, missing)

In [118]:
#
# todo: 
# variablen nur einmal lesen, und nicht für jedern convertierweg nochmal
# für alle methoden sammeln und dann das, was mehrfach da ist, aussortieren
# das übrige dann mit dem vergleichen, was schon im Datensatz steht -> auffüllen 
# fertig
#



def convert_missing(file, missing, destination: str = './converted'):
    with eua.CDMDataset(file) as data:
        df = data.to_dataframe(groups='observations_table', variables=['observed_variable', 'observation_value', 'date_time', 'z_coordinate', 'report_id', 'latitude', 'longitude'])
        
# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

    if 106 in missing or 107 in missing: # wd and/or ws is missing -> calculate it from u/v
        u = np.asarray(df[df.observed_variable == 104].observation_value)
        v = np.asarray(df[df.observed_variable == 105].observation_value)
        ws = np.sqrt(u ** 2 + v ** 2)
        wd = 90 - np.arctan2(-v, -u) * 180 / np.pi - 180.
        wd = np.where(wd > 0., wd, 360.+wd)
        return wd, ws
        #add
        #set conversion flag
    if 104 in missing or 105 in missing: # u and/or v is missing -> calculate it from wd and ws
        wd = np.asarray(df[df.observed_variable == 106].observation_value)
        ws = np.asarray(df[df.observed_variable == 107].observation_value)
        u = ws * np.cos(np.radians(wd))
        v = ws * np.sin(np.radians(wd)) 
        return u, v
        #add
        #set conversion flag
            
# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

    #
    # convert from every source and merge the rh outputs? maximize the searched variable -> removed elif and made if everywher
    #
    if 38 in missing: # rh is missing -> calculate it from 
        hums = [34, 36, 37, 39]
        missing_hum_vars = check_for_missing(file, checkvars = hums)
        for i in missing_hum_vars: hums.remove(i)
            
        if 34 in hums: # dew point depression
            t = df[df.observed_variable == 85]
            dpd = df[df.observed_variable == 34]

            cleaned = t[0:0]
            for i in dpd.report_id.drop_duplicates():
                inter = t[t.report_id == i]
                for j in dpd[dpd.report_id == i].z_coordinate:
                    cleaned = cleaned.append(inter[inter.z_coordinate == j])

            t = cleaned.observation_value.to_xarray()
            dpd = dpd.observation_value.to_xarray()

            rh = rasotools.met.convert.to_rh(t, dpd=dpd)
            return rh
            #add
            #set conversion flag
        if 36 in hums: # dew point
            t = df[df.observed_variable == 85]
            dp = df[df.observed_variable == 36]

            cleaned = t[0:0]
            for i in dp.report_id.drop_duplicates():
                inter = t[t.report_id == i]
                for j in dp[dp.report_id == i].z_coordinate:
                    cleaned = cleaned.append(inter[inter.z_coordinate == j])

            dpd = cleaned.observation_value.subtract(np.asarray(dp.observation_value))
            t = cleaned.observation_value.to_xarray()
            dpd = dpd.to_xarray()

            rh = rasotools.met.convert.to_rh(t,dpd=dpd)
            return rh
            #add
            #set conversion flag
        if 39 in hums: # specific humidity
            t = df[df.observed_variable == 85]
            sh = df[df.observed_variable == 39]

            cleaned = t[0:0]
            for i in sh.report_id.drop_duplicates():
                inter = t[t.report_id == i]
                for j in sh[sh.report_id == i].z_coordinate:
                    cleaned = cleaned.append(inter[inter.z_coordinate == j])

            p = cleaned.z_coordinate.to_xarray()
            sh = sh.observation_value.to_xarray()

            rh = rasotools.met.convert.to_rh(spec_humi=sh,press=p)
            return rh
            #add
            #set conversion flag
        else:
            print('-')
            print('NO HUMIDITY VARIABLE AVAILABLE - COULD NOT CONVERT')
            print('-')
            
# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

    #
    # convert from every source and merge the rh outputs? maximize the searched variable -> removed elif and made if everywher
    #
    if 34 in missing: # dew point depression is missing -> calculate it from
        hums = [38, 36, 37, 39]
        missing_hum_vars = check_for_missing(file, checkvars = hums)
        for i in missing_hum_vars: hums.remove(i)
            
        if 38 in hums and not 85 in missing: # relative humidity
            t = df[df.observed_variable == 85]
            rh = df[df.observed_variable == 38]

            cleaned = t[0:0]
            for i in rh.report_id.drop_duplicates():
                inter = t[t.report_id == i]
                for j in rh[rh.report_id == i].z_coordinate:
                    cleaned = cleaned.append(inter[inter.z_coordinate == j])

            t = cleaned.observation_value.to_xarray()
            p = cleaned.z_coordinate.to_xarray()
            rh = rh.observation_value.to_xarray()

            dpd = rasotools.met.convert.to_dpd(temp=t,press=p,rel_humi=rh)
            return dpd
            #add
            #set conversion flag
        if 36 in hums: # dew point
            t = df[df.observed_variable == 85]
            dp = df[df.observed_variable == 36]

            cleaned = t[0:0]
            for i in dp.report_id.drop_duplicates():
                inter = t[t.report_id == i]
                for j in dp[dp.report_id == i].z_coordinate:
                    cleaned = cleaned.append(inter[inter.z_coordinate == j])

            dpd = cleaned.observation_value.subtract(np.asarray(dp.observation_value))
            dpd = dpd.to_xarray()
            return dpd
            #add
            #set conversion flag
        if 39 in hums: # specific humidity
            t = df[df.observed_variable == 85]
            sh = df[df.observed_variable == 39]

            cleaned = t[0:0]
            for i in sh.report_id.drop_duplicates():
                inter = t[t.report_id == i]
                for j in sh[sh.report_id == i].z_coordinate:
                    cleaned = cleaned.append(inter[inter.z_coordinate == j])

            p = cleaned.z_coordinate.to_xarray()
            t = cleaned.observation_value.to_xarray()
            sh = sh.observation_value.to_xarray()

            dpd = rasotools.met.convert.to_dpd(spec_humi=sh,press=p,temp=t)
            return dpd
            #add
            #set conversion flag
        else:
            print('-')
            print('NO HUMIDITY VARIABLE AVAILABLE - COULD NOT CONVERT')
            print('-')
            
# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

    #
    # convert from every source and merge the rh outputs? maximize the searched variable -> removed elif and made if everywher
    #
    if 36 in missing: # dew point is missing -> calculate it from
        hums = [34, 38, 37, 39]
        missing_hum_vars = check_for_missing(file, checkvars = hums)
        for i in missing_hum_vars: hums.remove(i)
            
        if 38 in hums and not 85 in missing: # relative humidity
            t = df[df.observed_variable == 85]
            rh = df[df.observed_variable == 38]

            cleaned = t[0:0]
            for i in rh.report_id.drop_duplicates():
                inter = t[t.report_id == i]
                for j in rh[rh.report_id == i].z_coordinate:
                    cleaned = cleaned.append(inter[inter.z_coordinate == j])

            t = cleaned.observation_value.to_xarray()
            p = cleaned.z_coordinate.to_xarray()
            rh = rh.observation_value.to_xarray()

            dp = rasotools.met.convert.to_dewpoint(temp=t,press=p,rel_humi=rh)
            return dp
            #add
            #set conversion flag
        if 34 in hums: # dew point depression
            t = df[df.observed_variable == 85]
            dpd = df[df.observed_variable == 34]

            cleaned = t[0:0]
            for i in dpd.report_id.drop_duplicates():
                inter = t[t.report_id == i]
                for j in dpd[dpd.report_id == i].z_coordinate:
                    cleaned = cleaned.append(inter[inter.z_coordinate == j])

            dp = cleaned.observation_value.subtract(np.asarray(dpd.observation_value))
            dp = dp.to_xarray()
            return dp
            #add
            #set conversion flag
        if 39 in hums: # specific humidity
            t = df[df.observed_variable == 85]
            sh = df[df.observed_variable == 39]

            cleaned = t[0:0]
            for i in sh.report_id.drop_duplicates():
                inter = t[t.report_id == i]
                for j in sh[sh.report_id == i].z_coordinate:
                    cleaned = cleaned.append(inter[inter.z_coordinate == j])

            p = cleaned.z_coordinate.to_xarray()
            sh = sh.observation_value.to_xarray()
            dp = rasotools.met.convert.to_dewpoint(spec_humi=sh,press=p)
            return dp
            #add
            #set conversion flag
        else:
            print('-')
            print('NO HUMIDITY VARIABLE AVAILABLE - COULD NOT CONVERT')
            print('-')
            
# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

    #
    # convert from every source and merge the rh outputs? maximize the searched variable -> removed elif and made if everywher
    #
    if 39 in missing: # specific humidity is missing -> calculate it from
        hums = [34, 38, 37, 36]
        missing_hum_vars = check_for_missing(file, checkvars = hums)
        for i in missing_hum_vars: hums.remove(i)
            
        if 38 in hums and not 85 in missing: # relative humidity
            t = df[df.observed_variable == 85]
            rh = df[df.observed_variable == 38]

            cleaned = t[0:0]
            for i in rh.report_id.drop_duplicates():
                inter = t[t.report_id == i]
                for j in rh[rh.report_id == i].z_coordinate:
                    cleaned = cleaned.append(inter[inter.z_coordinate == j])

            t = cleaned.observation_value.to_xarray()
            p = cleaned.z_coordinate.to_xarray()
            rh = rh.observation_value.to_xarray()

            sh = rasotools.met.convert.to_sh(temp=t, press=p, rel_humi=rh)
            return sh
            #add
            #set conversion flag
        if 34 in hums: # dew point depression
            t = df[df.observed_variable == 85]
            dpd = df[df.observed_variable == 34]

            cleaned = t[0:0]
            for i in dpd.report_id.drop_duplicates():
                inter = t[t.report_id == i]
                for j in dpd[dpd.report_id == i].z_coordinate:
                    cleaned = cleaned.append(inter[inter.z_coordinate == j])

            t = cleaned.observation_value.to_xarray()
            p = cleaned.z_coordinate.to_xarray()
            dpd = dpd.observation_value.to_xarray()

            sh = rasotools.met.convert.to_sh(dpd=dpd, press=p, temp=t)
            return sh
            #add
            #set conversion flag
        if 36 in hums: # dew point
            t = df[df.observed_variable == 85]
            dp = df[df.observed_variable == 36]

            cleaned = t[0:0]
            for i in dp.report_id.drop_duplicates():
                inter = t[t.report_id == i]
                for j in dp[dp.report_id == i].z_coordinate:
                    cleaned = cleaned.append(inter[inter.z_coordinate == j])

            dpd = cleaned.observation_value.subtract(np.asarray(dp.observation_value))

            t = cleaned.observation_value.to_xarray()
            p = cleaned.z_coordinate.to_xarray()
            dpd = dpd.to_xarray()
            sh = rasotools.met.convert.to_sh(dpd=dpd, press=p, temp=t)
            return sh
            #add
            #set conversion flag
        else:
            print('-')
            print('NO HUMIDITY VARIABLE AVAILABLE - COULD NOT CONVERT')
            print('-')

In [119]:
convert_missing('/raid60/scratch/uli/resorted_files_correct/0-20000-0-11035_CEUAS_merged_v0.nc', missing=[38])

<xarray.DataArray 'rh' (index: 3974)>
array([0.730152, 0.623646, 0.645716, ..., 0.286004, 0.938464, 0.951918],
      dtype=float32)
Coordinates:
  * index    (index) int64 8906391 8906392 8906393 ... 10361354 10361355
Attributes:
    units:          1
    standard_name:  relativ_humidity
    long_name:      relative humidity
    esat:           HylandWexler
    origin:         t,dpd
    precision:      6

In [ ]:
convert_missing('/raid60/scratch/uli/resorted_files_correct/0-20000-0-11035_CEUAS_merged_v0.nc', missing=[36])

In [ ]:
convert_missing('/raid60/scratch/uli/resorted_files_correct/0-20000-0-11035_CEUAS_merged_v0.nc', missing=[34])

In [ ]:
convert_missing('/raid60/scratch/uli/resorted_files_correct/0-20000-0-11035_CEUAS_merged_v0.nc', missing=[39])

In [112]:
convert_missing('/raid60/scratch/uli/resorted_files_correct/0-20000-0-11035_CEUAS_merged_v0.nc', missing=[106])

(array([140.      , 120.      , 110.      , ..., 106.      , 106.      ,
        107.000015], dtype=float32),
 array([19.      , 22.6     , 15.399999, ...,  9.7     ,  9.3     ,
         9.1     ], dtype=float32))

In [ ]:
convert_missing('/raid60/scratch/uli/resorted_files_correct/0-20000-0-11035_CEUAS_merged_v0.nc', missing=[104])

In [113]:
with eua.CDMDataset('/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11035_CEUAS_merged_v0.nc') as data:
    df = data.to_dataframe(groups='observations_table', variables=['observed_variable', 'observation_value', 'z_coordinate', 'date_time', 'report_id'])
#     test = df[df.z_coordinate == 200.0]
    print(len(df[df.observed_variable == 106].observed_variable))
    print(len(df[df.observed_variable == 107].observed_variable))
#     test = test[test.date_time == '1910-04-12 08:29:00']
#     print(test)

4644756
4644725


In [115]:
with eua.CDMDataset('/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11035_CEUAS_merged_v0.nc') as data:
    df = data.to_dataframe(groups='observations_table', variables=['observed_variable', 'observation_value', 'z_coordinate', 'date_time', 'report_id'])
#     test = df[df.z_coordinate == 200.0]
    print(len(df[df.observed_variable == 85].observed_variable))


3573965


In [114]:
with eua.CDMDataset('/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11035_CEUAS_merged_v0.nc') as data:
    df = data.to_dataframe(groups='observations_table', variables=['observed_variable', 'observation_value', 'z_coordinate', 'date_time', 'report_id'])
#     test = df[df.z_coordinate == 200.0]
    print(len(df[df.observed_variable == 104].observed_variable))
    print(len(df[df.observed_variable == 105].observed_variable))
#     test = test[test.date_time == '1910-04-12 08:29:00']
#     print(test)

4546918
4546918


In [105]:
with eua.CDMDataset('/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11035_CEUAS_merged_v0.nc') as data:
    df = data.to_dataframe(groups='observations_table', variables=['observed_variable', 'observation_value', 'z_coordinate', 'date_time', 'report_id'])
    for i in df.report_id.drop_duplicates():
        test = df[df.report_id == i]
        for j in test.date_time.drop_duplicates():
            test2 = test[test.date_time == j]
            u = test2[test2.observation_value == 104]
            v = test2[test2.observation_value == 105]
            if(len(u) != len(v)):
                print(test2.date_time)
                print(test2.z_coordinate)
                print('--------')

141   1910-04-12 08:29:00
142   1910-04-12 08:29:00
143   1910-04-12 08:29:00
144   1910-04-12 08:29:00
145   1910-04-12 08:29:00
146   1910-04-12 08:29:00
147   1910-04-12 08:29:00
148   1910-04-12 08:29:00
149   1910-04-12 08:29:00
150   1910-04-12 08:29:00
151   1910-04-12 08:29:00
152   1910-04-12 08:29:00
153   1910-04-12 08:29:00
154   1910-04-12 08:29:00
155   1910-04-12 08:29:00
156   1910-04-12 08:29:00
157   1910-04-12 08:29:00
158   1910-04-12 08:29:00
159   1910-04-12 08:29:00
160   1910-04-12 08:29:00
161   1910-04-12 08:29:00
Name: date_time, dtype: datetime64[ns]
141     200.0
142     200.0
143     200.0
144     350.0
145     350.0
146     350.0
147     750.0
148     750.0
149     750.0
150    1250.0
151    1250.0
152    1250.0
153    1750.0
154    1750.0
155    1750.0
156    2250.0
157    2250.0
158    2250.0
159    2750.0
160    2750.0
161    2750.0
Name: z_coordinate, dtype: float32
--------


KeyboardInterrupt: 

In [95]:
convert_missing('/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11035_CEUAS_merged_v0.nc', missing=[104])

ValueError: operands could not be broadcast together with shapes (4644725,) (4644756,) 

In [92]:
convert_missing('/raid60/scratch/uli/resorted_files_correct/0-20000-0-11035_CEUAS_merged_v0.nc', missing=[34])

KeyboardInterrupt: 